In [1]:
import sys
import os
os.chdir("..")
import pandas as pd

In [2]:
train = pd.read_csv('data/feature/X_train.csv')

In [3]:
pd.set_option('display.max_columns', None)

In [4]:
missing_values = train.isnull().sum()
print(missing_values[missing_values > 0])

Series([], dtype: int64)
